In [ ]:
import pdfplumber as pp
import os
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import pandas as pd
import cv2
import numpy as np
import re

In [ ]:
folder_path = '..\\ListaVotacao'

In [ ]:
def rename_pdfs(folder_path, base_name='Pdf_ListaVotacao'):
    # Listar todos os arquivos no diretório especificado
    files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    
    # Ordenar os arquivos para garantir uma sequência consistente
    files.sort()

    # Iterar sobre os arquivos e renomear cada um
    for i, filename in enumerate(files, start=1):
        # Definir o novo nome usando o padrão 'Pdf_Lista_01', 'Pdf_Lista_02', etc.
        new_name = f"{base_name}_{i:02}.pdf"  # :02 garante que o número tenha dois dígitos
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_name)

        # Renomear o arquivo
        os.rename(old_path, new_path)
        print(f"Renomeado: {filename} -> {new_name}")



rename_pdfs(folder_path)

In [ ]:
# Extrair dados do pdf


def preprocess_image(image):
    # Converter para escala de cinza
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    
    # Aplicar binarização (thresholding)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    
    # Remover ruído (opcional)
    denoised = cv2.fastNlMeansDenoising(binary, None, 30, 7, 21)
    
    # Converter de volta para uma imagem PIL para usar com pytesseract
    processed_image = Image.fromarray(denoised).rotate(270, expand=True)
    
    return processed_image


def extract_text_with_ocr(pdf_path):
    text_dict = {}
    # Converter cada página do PDF em imagem
    pages = convert_from_path(pdf_path, dpi=850, first_page=1, last_page=10)
    for i, page in enumerate(pages, start=1):
        # Pré-processar a imagem
        preprocessed_page = preprocess_image(page)
        
        # Usar OCR para extrair texto da imagem processada
        page_text = pytesseract.image_to_string(preprocessed_page)
        
        # Aplicar a substituição, se necessário
        # page_text = re.sub(r'-\s*[A-Za-z]+/[A-Za-z]+\s+\d{2,5}', '- 000.000.000-00', page_text)
        
        # Dividir o texto da página em linhas
        linhas = page_text.split("\n")
        
        # Filtrar para remover as duas primeiras linhas e a última
        linhas_filtradas = linhas
        
        # Adicionar as linhas filtradas ao dicionário com a página como chave
        text_dict[f'pagina_{i}'] = linhas_filtradas
    
    return text_dict

def remover_itens_em_branco(dados):
    # Itera sobre cada chave e valor no dicionário
    for pagina, conteudo in dados.items():
        # Filtra os itens em branco
        dados[pagina] = [item for item in conteudo if item]
    return dados


# Chamando a função para remover itens em branco

# Caminho do PDF
pdf_path = f'{folder_path}/Pdf_ListaVotacao_01.pdf'
text = extract_text_with_ocr(pdf_path)
text = remover_itens_em_branco(text)

In [ ]:
################## EXTRAIR NOME E CPF #########################

# Regex para capturar linhas no formato "NOME - CPF/OUTRO"
nome_cpf_pattern = re.compile(r'^.+? - .+$')

# Lista para armazenar os nomes extraídos
nomes = []

# Iterar sobre as linhas processadas
for line in resultados:
    match = nome_cpf_pattern.search(line)
    if match:
        # Capturar o nome completo com o padrão e remover espaços e hífens desnecessários
        nome = match.group().rstrip(' -')
        nomes.append(nome)

# Exibindo os nomes de interesse (exemplo: do índice 60 ao 90)
for nome in nomes:
    print(nome)

# Total de nomes encontrados
print(f"Total de nomes encontrados: {len(nomes)}")

In [ ]:
####################### EXTRAIR CLASSE ######################

classe_pattern = re.compile(r'\bTrabalhista|Quirografarios|ME e EPP\b')


# Lista para armazenar as classes encontradas
classes = []

for page, lines in text.items():
    for line in lines:
        match = classe_pattern.search(line)
        if match:
            classe = match.group()
            classes.append(classe)

# Exibindo o resultado
print(classes)
print(f"Total de ocorrências da classe 'Trabalhista': {len(classes)}")

In [ ]:
####################### EXTRAIR VOTO ####################

voto_pattern = re.compile(r'\b(Rejeita|Abstém|Aprova|Abstem|Abstéem)\b')

votos = []
for page, lines in text.items():
    for line in lines:
        match = voto_pattern.search(line)
        if match:
            voto = match.group()
            votos.append(voto)

# Exibindo o resultado
print(votos)
print(f"Total de nomes encontrados: {len(votos)}")

In [ ]:
##################### EXTRAIR DATAS ####################

data_patter = re.compile(r'\b\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\b')

datas = []
for page, lines in text.items():
    for line in lines:
        match = data_patter.search(line)
        if match:
            data = match.group()
            datas.append(data)

# Exibindo o resultado
print(datas)
print(f"Total de nomes encontrados: {len(datas)}")

In [ ]:
########################## EXPORTAR TABELA #########################

dados = {
    "Nome": nomes, 
    "Classe": classes, 
    "Voto": votos, 
    "Data":datas
    }

tabela = pd.DataFrame(dados)


tabela.to_excel('..\\TabelaVotacaoFull.xlsx')